# 20241127
---

## 문제 1 (23년 6회)

- 도수 분포와 기대 분포의 차이가 있는지 검정하는 문제 (카이제곱 적합도 검정)

In [62]:
import pandas as pd

df = pd.read_csv('./datasets/P230605.csv', encoding='euckr')

# (1) 부작용 없음 데이터를 0~1 사이의 확률로 출력하기
df = df.groupby('코드').size().reset_index(name='건수')  # 그룹화 한 후, 집계한 결과의 컬럼명을 '건수'로 설정
print(df)

""" 출력 결과
   코드   건수
0   1   15
1   2   24
2   3   15
3   4  199
"""

total_count = 253
df['확률'] = df['건수'] / total_count
print(df)

""" 출력 결과
   코드   건수     비율
0   1   15  0.059289
1   2   24  0.094862
2   3   15  0.059289
3   4  199  0.786561
"""

cond = df['코드'] == 4
answer1 = round(df[cond]['확률'], 3)

print(answer1[3])

# (2) 카이제곱 검정으로 검정 통계량 출력하기

obs = df['건수']
df['비율'] = [0.05, 0.1, 0.05, 0.8]
df['예상값'] = df['비율'] * total_count

print(df)

""" 출력 결과
   코드   건수        확률    비율     예상값
0   1   15  0.059289  0.05   12.65
1   2   24  0.094862  0.10   25.30
2   3   15  0.059289  0.05   12.65
3   4  199  0.786561  0.80  202.40
"""

obs = df['건수']    # observed
exp = df['예상값']  # expected

from scipy.stats import chisquare

result = chisquare(obs, f_exp=exp)
print(result)

answer2 = round(result.statistic, 3)
print(answer2)

# (3) 유의 확률(p-value) 구하기
answer3 = round(result.pvalue, 3)
print(answer3)

   코드   건수
0   1   15
1   2   24
2   3   15
3   4  199
   코드   건수        확률
0   1   15  0.059289
1   2   24  0.094862
2   3   15  0.059289
3   4  199  0.786561
0.787
   코드   건수        확률    비율     예상값
0   1   15  0.059289  0.05   12.65
1   2   24  0.094862  0.10   25.30
2   3   15  0.059289  0.05   12.65
3   4  199  0.786561  0.80  202.40
Power_divergenceResult(statistic=0.9970355731225298, pvalue=0.801969260894451)
0.997
0.802


## 문제 2 (23년 6회)

In [76]:
import pandas as pd

df = pd.read_csv('./datasets/P230606.csv', encoding='euckr')

print(df.info())

""" 출력 결과
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   O3           111 non-null    int64  
 1   Solar        111 non-null    int64  
 2   Wind         111 non-null    float64
 3   Temperature  111 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 3.6 KB
"""

# (1) 오존 농도 변수에 대한 회귀 계수 추정값 출력
import statsmodels.api as sm

formula = 'Temperature ~ O3 + Solar + Wind'
model = sm.OLS.from_formula(formula, data=df).fit()

summary = model.summary()
print(summary)

""" 출력 결과
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Temperature   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.486
Method:                 Least Squares   F-statistic:                     35.65
Date:                Wed, 27 Nov 2024   Prob (F-statistic):           4.73e-16
Time:                        17:09:17   Log-Likelihood:                -368.79
No. Observations:                 111   AIC:                             745.6
Df Residuals:                     107   BIC:                             756.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     72.4186      3.216     22.522      0.000      66.044      78.793
O3             0.1720      0.026      6.516      0.000       0.120       0.224
Solar          0.0073      0.008      0.948      0.345      -0.008       0.022
Wind          -0.3229      0.233     -1.384      0.169      -0.785       0.139
==============================================================================
Omnibus:                        6.409   Durbin-Watson:                   1.170
Prob(Omnibus):                  0.041   Jarque-Bera (JB):                6.615
Skew:                          -0.591   Prob(JB):                       0.0366
Kurtosis:                       2.817   Cond. No.                     1.05e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

answer1 = round(model.params['O3'], 3)
print(answer1)

""" 출력 결과
0.172
"""

# (2) 오존 농도, 일사량이 고정일 때 풍속이 증가함에 따라 온도가 낮아진다는 것을 검증했다. T-검증 값의 유의 확률(p-value) 구하기
answer2 = round(model.pvalues['Wind'], 3)
print(answer2)

""" 출력 결과
0.0
"""

# (3) 어떤 날이 오존 농도 10, 일사량 90, 풍속 20일 때 온도의 예측값 구하기
new_df = pd.DataFrame({
    # 💡 ⁡⁣⁢⁣상수항을 따로 추가하지 않아도 된다 (위에서 추가했으므로) ⁡
    'O3': [10], 
    'Solar': [90],
    'Wind': [20]
})

result = model.predict(new_df)
print(result)

""" 출력 결과
0    68.334156
dtype: float64
"""

answer3 = round(result[0], 3)
print(answer3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   O3           111 non-null    int64  
 1   Solar        111 non-null    int64  
 2   Wind         111 non-null    float64
 3   Temperature  111 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 3.6 KB
None
                            OLS Regression Results                            
Dep. Variable:            Temperature   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.486
Method:                 Least Squares   F-statistic:                     35.65
Date:                Wed, 27 Nov 2024   Prob (F-statistic):           4.73e-16
Time:                        17:36:30   Log-Likelihood:                -368.79
No. Observations:                 111   AIC:                             745.6
Df Residuals:                     10

## 문제 3 (23년 7회)

In [98]:
import numpy as np
import pandas as pd

# 가상 데이터 생성
np.random.seed(0)  # 재현성을 위해 시드 설정
n_samples = 100

# 독립 변수(v1 ~ v5) 및 종속 변수(Target) 생성
data = {
    'v1': np.random.rand(n_samples) * 100,
    'v2': np.random.rand(n_samples) * 50,
    'v3': np.random.rand(n_samples) * 200,
    'v4': np.random.rand(n_samples) * 80,
    'v5': np.random.rand(n_samples) * 10,
}

# Target 변수는 v1, v2, v3에 영향을 받도록 생성
data['Target'] = (
    3 * data['v1'] - 2 * data['v2'] + 0.5 * data['v3'] +
    np.random.randn(n_samples) * 20  # 노이즈 추가
)

# DataFrame으로 변환
df = pd.DataFrame(data)


# (1) 선형 관계가 가장 큰 변수 찾아 상관 계수 구하기
corr_table = df.corr(numeric_only=True)
print(corr_table)

""" 출력 결과
              v1        v2        v3        v4        v5    Target
v1      1.000000 -0.066107 -0.036512  0.021895 -0.005389  0.872265
v2     -0.066107  1.000000 -0.124047 -0.061987 -0.114401 -0.383313
v3     -0.036512 -0.124047  1.000000 -0.115117  0.062719  0.310146
v4      0.021895 -0.061987 -0.115117  1.000000  0.033356 -0.011590
v5     -0.005389 -0.114401  0.062719  0.033356  1.000000  0.036221
Target  0.872265 -0.383313  0.310146 -0.011590  0.036221  1.000000
"""

target_corr = corr_table['Target'].sort_values(ascending=False)
print(target_corr)

""" 출력 결과
Target    1.000000
v1        0.872265
v3        0.310146
v5        0.036221
v4       -0.011590
v2       -0.383313

Name: Target, dtype: float64
"""

answer1 = target_corr[1]
print(answer1)

""" 출력 결과
0.8722647988549976
"""

# (2) Target 변수를 종속 변수로 하여 다중 선형 회귀 모델링을 진행했을 때, V2 컬럼의 회귀 계수 구하기
import statsmodels.api as sm

formula = "Target ~ v1 + v2 + v3 + v4 + v5"

model = sm.OLS.from_formula(formula, data=df).fit()
summary = model.summary()

print(summary)

""" 출력 결과
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Target   R-squared:                       0.960
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                     447.0
Date:                Wed, 27 Nov 2024   Prob (F-statistic):           7.20e-64
Time:                        17:45:02   Log-Likelihood:                -443.47
No. Observations:                 100   AIC:                             898.9
Df Residuals:                      94   BIC:                             914.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.2852      9.174      0.358      0.721     -14.929      21.500
v1             3.0453      0.073     41.575      0.000       2.900       3.191
v2            -2.1320      0.155    -13.754      0.000      -2.440      -1.824
v3             0.5132      0.036     14.452      0.000       0.443       0.584
v4            -0.0582      0.093     -0.624      0.534      -0.244       0.127
v5            -0.3720      0.705     -0.528      0.599      -1.771       1.027
==============================================================================
Omnibus:                        2.625   Durbin-Watson:                   1.810
Prob(Omnibus):                  0.269   Jarque-Bera (JB):                2.212
Skew:                          -0.361   Prob(JB):                        0.331
Kurtosis:                       3.101   Cond. No.                         577.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""


answer2 = model.params['v2']
print(answer2)

""" 출력 결과
-2.131974490127022
"""


# (3) 회귀 계수들이 가지는 p-값들 중 최대값 구하기
pvalues = model.pvalues
print(pvalues)

""" 출력 결과
Intercept    7.210653e-01
v1           2.577718e-62
v2           3.094160e-24
v3           1.290258e-25
v4           5.344372e-01
v5           5.988162e-01
dtype: float64
"""

sorted_pvalues = pvalues.sort_values(ascending=False)
print(sorted_pvalues)


""" 출력 결과
Intercept    7.210653e-01
v5           5.988162e-01
v4           5.344372e-01
v2           3.094160e-24
v3           1.290258e-25
v1           2.577718e-62
dtype: float64
"""

answer3 = sorted_pvalues[2]
print(answer3)

""" 출력 결과
0.5344372455396594
"""

              v1        v2        v3        v4        v5    Target
v1      1.000000 -0.066107 -0.036512  0.021895 -0.005389  0.872265
v2     -0.066107  1.000000 -0.124047 -0.061987 -0.114401 -0.383313
v3     -0.036512 -0.124047  1.000000 -0.115117  0.062719  0.310146
v4      0.021895 -0.061987 -0.115117  1.000000  0.033356 -0.011590
v5     -0.005389 -0.114401  0.062719  0.033356  1.000000  0.036221
Target  0.872265 -0.383313  0.310146 -0.011590  0.036221  1.000000
Target    1.000000
v1        0.872265
v3        0.310146
v5        0.036221
v4       -0.011590
v2       -0.383313
Name: Target, dtype: float64
0.8722647988549976
                            OLS Regression Results                            
Dep. Variable:                 Target   R-squared:                       0.960
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                     447.0
Date:                Wed, 27 Nov 2024   Prob (F-stati

C:\Users\kss34\AppData\Local\Temp\ipykernel_37052\1978882569.py:55: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer1 = target_corr[1]
C:\Users\kss34\AppData\Local\Temp\ipykernel_37052\1978882569.py:142: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer3 = sorted_pvalues[2]


## 문제 4 (23년 7회)

In [139]:
import pandas as pd

df = pd.read_csv('./datasets/P240604.csv')

print(df.info())

""" 출력 결과
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   age                         200 non-null    int64  
 1   calls                       200 non-null    int64  
 2   sms_usage                   200 non-null    int64  
 3   gender                      200 non-null    int64  
 4   monthly_charge              200 non-null    float64
 5   churn                       200 non-null    int64  
 6   contract_length             200 non-null    int64  
 7   data_usage                  200 non-null    float64
 8   internet_usage              200 non-null    int64  
 9   customer_support_calls      200 non-null    int64  
 10  payment_method_Credit Card  200 non-null    int64  
 11  payment_method_PayPal       200 non-null    int64  
 12  region_North                200 non-null    int64  
 13  region_South                200 non-null    int64  
 14  region_West                 200 non-null    int64  
dtypes: float64(2), int64(13)
memory usage: 23.6 KB
"""

# (1) 로지스틱 회귀를 수행하고, 유의하지 않은 변수 개수 구하기
import statsmodels.api as sm

# 회귀 분석
## - 귀무 가설 : 해당 변수는 유의하지 않다. (p-value >= 0.05)
## - 대립 가설 : 해당 변수는 유의하다. (p-value < 0.05)


target = 'churn'

columns = df.columns
columns = columns.drop(target)

print(columns)

""" 출력 결과
Index(['age', 'calls', 'sms_usage', 'gender', 'monthly_charge',
       'contract_length', 'data_usage', 'internet_usage',
       'customer_support_calls', 'payment_method_Credit_Card',
       'payment_method_PayPal', 'region_North', 'region_South', 'region_West'],
      dtype='object')
"""

result = ""
for index, col in enumerate(columns):
    if index == 0:
        result += f"{col}"
    else:
        result += f" + {col}"

print(result)

""" 출력 결과
age + calls + sms_usage + gender + monthly_charge + contract_length + data_usage + internet_usage + customer_support_calls + payment_method_Credit Card + payment_method_PayPal + region_North + region_South + region_West

"""

formula = f"{target} ~ {result}"
print(formula)

""" 출력 결과
churn ~ age + calls + sms_usage + gender + monthly_charge + contract_length + data_usage + internet_usage + customer_support_calls + payment_method_Credit Card + payment_method_PayPal + region_North + region_South + region_West
"""

import statsmodels.api as sm

model = sm.Logit.from_formula(formula, data=df).fit()
summary = model.summary()

print(summary)

""" 출력 결과
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  churn   No. Observations:                  200
Model:                          Logit   Df Residuals:                      185
Method:                           MLE   Df Model:                           14
Date:                Wed, 27 Nov 2024   Pseudo R-squ.:                  0.6747
Time:                        18:09:32   Log-Likelihood:                -18.139
converged:                       True   LL-Null:                       -55.754
Covariance Type:            nonrobust   LLR p-value:                 2.148e-10
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                    -48.4932     13.372     -3.626      0.000     -74.702     -22.285
age                            0.3010      0.086      3.490      0.000       0.132       0.470
calls                          0.4003      0.130      3.077      0.002       0.145       0.655
sms_usage                     -0.0186      0.010     -1.769      0.077      -0.039       0.002
gender                         2.2272      1.242      1.793      0.073      -0.208       4.662
monthly_charge                 0.2638      0.079      3.346      0.001       0.109       0.418
contract_length                0.0824      0.046      1.812      0.070      -0.007       0.172
data_usage                     0.0398      0.105      0.380      0.704      -0.165       0.245
internet_usage                -0.2261      0.402     -0.563      0.574      -1.014       0.562
customer_support_calls        -0.7655      0.451     -1.697      0.090      -1.649       0.118
payment_method_Credit_Card    -1.1847      1.245     -0.952      0.341      -3.624       1.255
payment_method_PayPal         -2.5692      1.559     -1.648      0.099      -5.625       0.487
region_North                  -1.0847      1.325     -0.819      0.413      -3.681       1.512
region_South                  -0.0380      1.468     -0.026      0.979      -2.916       2.840
region_West                    0.0739      1.739      0.042      0.966      -3.334       3.482
==============================================================================================

Possibly complete quasi-separation: A fraction 0.49 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""


pvalues = model.pvalues
print(pvalues)

""" 출력 결과
Intercept                     0.000287
age                           0.000484
calls                         0.002091
sms_usage                     0.076855
gender                        0.073040
monthly_charge                0.000821
contract_length               0.070000
data_usage                    0.704035
internet_usage                0.573750
customer_support_calls        0.089623
payment_method_Credit_Card    0.341191
payment_method_PayPal         0.099402
region_North                  0.412949
region_South                  0.979358
region_West                   0.966111
dtype: float64
"""

answer1 = 0
bad_col = []
alpha = 0.05    # 유의 수준

for idx, value in enumerate(pvalues.values):
    # 상수항 제외
    if idx == 0:
        continue

    # 귀무 가설 : 해당 변수는 유의하지 않다.
    # -> 귀무 가설을 채택 (p-value >= 유의수준) 하는 변수 선택
    if value >= alpha:
        answer1 += 1
        bad_col.append(pvalues.index[idx])

print(bad_col)

""" 출력 결과
['sms_usage', 'gender', 'contract_length', 'data_usage', 'internet_usage', 'customer_support_calls', 'payment_method_Credit_Card', 'payment_method_PayPal', 'region_North', 'region_South', 'region_West']
"""

print(answer1)

""" 출력 결과
11
"""

# (2) 유의미한 변수만을 독립 변수로 하여 로지스틱 회귀를 다시 수행하고, 회귀 계수의 평균 구하기
columns = df.columns.drop('churn')
print(columns)

""" 출력 결과
['age', 'calls', 'sms_usage', 'gender', 'monthly_charge', 'contract_length', 'data_usage', 'internet_usage', 'customer_support_calls', 'payment_method_Credit_Card', 'payment_method_PayPal', 'region_North', 'region_South', 'region_West']
"""

good_col = []

for col in columns:
    # 유의하지 않은 변수 제외하기
    if col in bad_col:
        continue
    else:
        good_col.append(col)

print(good_col)

""" 출력 결과
['age', 'calls', 'monthly_charge']
"""

result = ""
for index, col in enumerate(good_col):
    if index == 0:
        result += f"{col}"
    else:
        result += f" + {col}"

print(result)

""" 출력 결과
age + calls + monthly_charge
"""

formula = f"{target} ~ {result}"

model = sm.Logit.from_formula(formula, data=df).fit()
summary = model.summary()
print(summary)

""" 출력 결과
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  churn   No. Observations:                  200
Model:                          Logit   Df Residuals:                      196
Method:                           MLE   Df Model:                            3
Date:                Wed, 27 Nov 2024   Pseudo R-squ.:                  0.5391
Time:                        18:23:05   Log-Likelihood:                -25.696
converged:                       True   LL-Null:                       -55.754
Covariance Type:            nonrobust   LLR p-value:                 5.554e-13
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        -40.2967      9.192     -4.384      0.000     -58.312     -22.281
age                0.2045      0.050      4.060      0.000       0.106       0.303
calls              0.2977      0.080      3.718      0.000       0.141       0.455
monthly_charge     0.1949      0.052      3.770      0.000       0.094       0.296
==================================================================================

Possibly complete quasi-separation: A fraction 0.28 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

coefs = model.params   # params 를 이용하여 회귀 계수 값들을 가져온다.
print(coefs)

""" 출력 결과
Intercept        -40.296671
age                0.204457
calls              0.297719
monthly_charge     0.194856
dtype: float64
"""

# 💡 ⁡⁢⁣⁣중요! "상수항" 까지 <포함> 시켜서 평균을 구한다.⁡
answer2 = coefs.mean()
print(answer2)

""" 출력 결과
-9.899909683457247
"""

# (3) (1), (2)에서 얻은 회귀식을 바탕으로, calls 변수가 5증가할 때 오즈비는 몇 배 증가하는지 구하기
import numpy as np

answer3 = np.exp(5 * coefs['calls'])    # 오즈비 = np.exp(회귀계수_변수)
print(answer3)


""" 출력 결과
4.430865146509951
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   age                         200 non-null    int64  
 1   calls                       200 non-null    int64  
 2   sms_usage                   200 non-null    int64  
 3   gender                      200 non-null    int64  
 4   monthly_charge              200 non-null    float64
 5   churn                       200 non-null    int64  
 6   contract_length             200 non-null    int64  
 7   data_usage                  200 non-null    float64
 8   internet_usage              200 non-null    int64  
 9   customer_support_calls      200 non-null    int64  
 10  payment_method_Credit_Card  200 non-null    int64  
 11  payment_method_PayPal       200 non-null    int64  
 12  region_North                200 non-null    int64  
 13  region_South                200 non

## 문제 5 (24년 8회)

In [ ]:
import pandas as pd

df = pd.read_csv('./datasets/P240605.csv')

print(df.info())

""" 출력 결과
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PIQ              300 non-null    float64
 1   brain_size       300 non-null    float64
 2   height           300 non-null    float64
 3   weight           300 non-null    float64
 4   education_level  300 non-null    int64  
 5   reading_habits   300 non-null    int64  
 6   age              300 non-null    int64  
dtypes: float64(4), int64(3)
memory usage: 16.5 KB
"""


# (1) 다중 선형 회귀 분석을 적용하여 각 독립 변수의 회귀 계수를 구하고, 가장 유의미한 변수의 회귀 계수 구하기
import statsmodels.api as sm

formula = 'PIQ ~ brain_size + height + weight + education_level + reading_habits + age'

model = sm.OLS.from_formula(formula, data=df).fit()
summary = model.summary()

print(summary)

""" 출력 결과
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    PIQ   R-squared:                       0.803
Model:                            OLS   Adj. R-squared:                  0.799
Method:                 Least Squares   F-statistic:                     198.5
Date:                Wed, 27 Nov 2024   Prob (F-statistic):          4.16e-100
Time:                        18:40:43   Log-Likelihood:                -910.43
No. Observations:                 300   AIC:                             1835.
Df Residuals:                     293   BIC:                             1861.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          40.9026      6.091      6.715      0.000      28.915      52.890
brain_size          0.0528      0.029      1.820      0.070      -0.004       0.110
height              0.0203      0.029      0.701      0.484      -0.037       0.077
weight             -0.0001      0.025     -0.005      0.996      -0.049       0.049
education_level    10.0616      0.372     27.079      0.000       9.330      10.793
reading_habits      5.0042      0.359     13.927      0.000       4.297       5.711
age                -0.4724      0.024    -19.384      0.000      -0.520      -0.424
==============================================================================
Omnibus:                        2.380   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.304   Jarque-Bera (JB):                1.951
Skew:                          -0.055   Prob(JB):                        0.377
Kurtosis:                       2.621   Cond. No.                     4.29e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.29e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""


coeffs = model.params
print(coeffs)

""" 출력 결과
Intercept          40.902575
brain_size          0.052773
height              0.020295
weight             -0.000134
education_level    10.061623
reading_habits      5.004246
age                -0.472388

dtype: float64
"""

pvalues = model.pvalues
print(pvalues)

""" 출력 결과
Intercept          9.734357e-11
brain_size         6.976485e-02
height             4.838630e-01
weight             9.957198e-01
education_level    9.675690e-82
reading_habits     3.507343e-34
age                1.940882e-54

dtype: float64
"""

## 회귀 분석
### - 귀무 가설 : 해당 변수는 유의하지 않다. (pvalue > 유의수준)
### - 대립 가설 : 해당 변는는 유의하다. (pvalue <= 유의수준)
### 가장 유의미하다 => pvalue 값이 제일 작은 것 찾기

pvalues_sorted = pvalues.sort_values(ascending=True)
print(pvalues_sorted)

""" 출력 결과
education_level    9.675690e-82 (v)
age                1.940882e-54
reading_habits     3.507343e-34
Intercept          9.734357e-11
brain_size         6.976485e-02
height             4.838630e-01
weight             9.957198e-01
dtype: float64
"""

target_column = pvalues_sorted.index[0]
print(target_column)

""" 출력 결과
education_level
"""

answer1 = coeffs[target_column]
print(answer1)

""" 출력 결과
10.061623490668252
"""

# (2) 테스트 데이터를 이용하여 결정계수 값 구하기
answer2 = model.rsquared
print(answer2)

""" 출력 결과
0.8025977310921423
"""

# (3) 다음과 같이 참가자 데이터가 주어 졌을 때, 다중 선형 회귀를 이용하여 각 참가자의 PIQ 예측하기
## brain_size = 150, height = 190, weight = 90, education_level = 3, reading_habits = 2, age = 15
new_data = pd.DataFrame({
    'brain_size': [150],
    'height': [190],
    'weight': [90],
    'education_level': [3],
    'reading_habits': [2],
    'age': [15],
})

result = model.predict(new_data)

answer3 = result.values[0]

print(answer3)

""" 결과 출력
85.77007654283213
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PIQ              300 non-null    float64
 1   brain_size       300 non-null    float64
 2   height           300 non-null    float64
 3   weight           300 non-null    float64
 4   education_level  300 non-null    int64  
 5   reading_habits   300 non-null    int64  
 6   age              300 non-null    int64  
dtypes: float64(4), int64(3)
memory usage: 16.5 KB
None
                            OLS Regression Results                            
Dep. Variable:                    PIQ   R-squared:                       0.803
Model:                            OLS   Adj. R-squared:                  0.799
Method:                 Least Squares   F-statistic:                     198.5
Date:                Wed, 27 Nov 2024   Prob (F-statistic):          4.16e-100
Time:                        18